In [2]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd
import shapely
from shapely.ops import nearest_points
from tqdm import tqdm, trange
import kml2geojson
import json
import math
from shapely.ops import cascaded_union
from shapely.geometry import Point, LineString
from shapely.geometry import mapping

In [4]:
import gpxpy 
import gpxpy.gpx

In [5]:
def nearest(row, geom_union, df1, df2, geom1_col='geometry', geom2_col='geometry', src_column=None):
    """Find the nearest point and return the corresponding value from specified column."""
    # Find the geometry that is closest
    nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
    # Get the corresponding value from df2 (matching is based on the geometry)
    value = df2[nearest][src_column].get_values()[0]
    return value

In [8]:
File_to_matrix = pd.read_csv('File_to_matrix.csv',sep=';', encoding='cp1251')
File_to_matrix = File_to_matrix[['время','дата обкатки','наименование маршрута','Stop_id','ID рейса','Геогр. широта','Геогр. долгота']]
File_to_matrix = File_to_matrix.rename(columns = {'наименование маршрута':'route_name', 'ID рейса':'line_id','Stop_id':'stop_id', 'Геогр. широта':'lat','Геогр. долгота':'lon'})
geometry = [Point(xy) for xy in zip(File_to_matrix.lon, File_to_matrix.lat)]
File_to_matrix = File_to_matrix.drop(['lon', 'lat'], axis=1)
File_to_matrix = gpd.GeoDataFrame(File_to_matrix, geometry=geometry)
File_to_matrix.crs = {"init": "EPSG:4326"}
File_to_matrix=File_to_matrix.to_crs({"init": "EPSG:32637"})
File_to_matrix

,время,дата обкатки,route_name,stop_id,line_id,geometry
0,9:48:07,14.11.2018,162,205,1,POINT (397458.7428795043 6018721.033645171)
1,9:48:39,14.11.2018,162,216,1,POINT (398607.2394801023 6017790.4752884)
2,9:49:11,14.11.2018,162,219,1,POINT (398807.6971361094 6017317.89482891)
3,9:49:43,14.11.2018,162,223,1,POINT (399174.9709620521 6017065.051377545)
4,9:50:52,14.11.2018,162,226,1,POINT (399470.4523100491 6016862.450741538)
5,9:54:28,14.11.2018,162,235,1,POINT (400026.2145460442 6016497.032422334)
6,9:55:45,14.11.2018,162,245,1,POINT (400492.3935259858 6015413.478947994)
7,9:56:32,14.11.2018,162,249,1,POINT (400825.5394195329 6015037.967031768)
8,9:57:39,14.11.2018,162,254,1,POINT (401104.8220987372 6014369.907290767)
9,10:01:28,14.11.2018,162,258,1,POINT (401294.0310750684 6013818.222019517)


In [9]:
df_of_waypoints = pd.DataFrame()
list2 = []
gpx_file = open('1_back.gpx', 'r') 
gpx = gpxpy.parse(gpx_file)
for waypoint in gpx.waypoints:
    list2.append('({0},{1})_{2})'.format(waypoint.latitude, waypoint.longitude, waypoint.time))
df_of_waypoints['text'] = list2

df_of_waypoints['coord'] = df_of_waypoints['text'].str.split('_',n=1, expand=True)[0]
df_of_waypoints['times'] = df_of_waypoints['text'].str.split('_',n=1, expand=True)[1]
df_of_waypoints['lat'] = df_of_waypoints['coord'].str.split(',',n=1, expand=True)[0]
df_of_waypoints['lat'] = df_of_waypoints.lat.str.replace("(", "")
df_of_waypoints['lat'] = df_of_waypoints['lat'].astype(float)
df_of_waypoints['lon'] = df_of_waypoints['coord'].str.split(',',n=1, expand=True)[1]
df_of_waypoints['lon'] = df_of_waypoints.lon.str.replace(")", "")
df_of_waypoints['lon'] = df_of_waypoints['lon'].astype(float)
geometry = [Point(xy) for xy in zip(df_of_waypoints.lon, df_of_waypoints.lat)]
df_of_waypoints = df_of_waypoints.drop(['coord','text','lon', 'lat'], axis=1)
df_of_waypoints = gpd.GeoDataFrame(df_of_waypoints, geometry=geometry)
df_of_waypoints.crs = {"init": "EPSG:4326"}
df_of_waypoints=df_of_waypoints.to_crs({"init": "EPSG:32637"})

df_of_waypoints['date'], df_of_waypoints['time'] = df_of_waypoints.times.str.split(' ',1).str
df_of_waypoints['time'] = df_of_waypoints.time.str.split('+',1,expand=True)
df_of_waypoints['hour'], df_of_waypoints['minuts'] = df_of_waypoints.time.str.split(':',1).str
df_of_waypoints['minuts'], df_of_waypoints['sek'] = df_of_waypoints.minuts.str.split(':',1).str
df_of_waypoints['hour'] = df_of_waypoints['hour'].astype(np.int64)
df_of_waypoints['minuts'] = df_of_waypoints['minuts'].astype(np.int64)
df_of_waypoints['sek'] = df_of_waypoints['sek'].astype(np.int64)
df_of_waypoints = df_of_waypoints.drop(['times'], axis=1)
df_of_waypoints = df_of_waypoints.reset_index().rename(columns = {'index':'waypoint_id'})
df_of_waypoints

,waypoint_id,geometry,date,time,hour,minuts,sek
0,0,POINT (413338.8373265794 6000084.834942986),2018-11-15,13:03:37,13,3,37
1,1,POINT (412063.7269404483 6000415.780505657),2018-11-15,13:04:38,13,4,38
2,2,POINT (411616.364319443 6000696.993979718),2018-11-15,13:05:39,13,5,39
3,3,POINT (411036.5929786218 6001369.777705503),2018-11-15,13:08:34,13,8,34
4,4,POINT (411054.171402988 6001746.283179988),2018-11-15,13:10:52,13,10,52
5,5,POINT (410994.1696260434 6002211.762544386),2018-11-15,13:12:16,13,12,16
6,6,POINT (411246.0000383027 6002777.29775908),2018-11-15,13:13:25,13,13,25
7,7,POINT (411318.6858865474 6003031.989449234),2018-11-15,13:14:43,13,14,43
8,8,POINT (411124.2751452824 6003374.290677111),2018-11-15,13:15:07,13,15,7
9,9,POINT (411298.2956411383 6003683.123225031),2018-11-15,13:16:30,13,16,30


In [10]:
tracks = pd.DataFrame()
list1 = []
list3 = []
gpx_file = open('1_back.gpx', 'r') 
for track in gpx.tracks:  
    for segment in track.segments:
        for point in segment.points:
            list1.append('({0},{1})_{2}'.format(point.latitude, point.longitude, point.time))
            list3.append(track.description)  
tracks['text'] =list1 
tracks['text_id'] =list3  

df = tracks
df['line_id'] = df['text_id'].str.split(',',n=1, expand=True)[0]
df['line_id'] = df['line_id'].str.split('D',n=1, expand=True)[1]
df['direction_text'] = df['text_id'].str.split(',',n=1, expand=True)[1]
df['direction'] = df['direction_text'].str.split(':',n=1, expand=True)[1]
df = df.drop(['text_id','direction_text'], axis=1)

df['coord'] = df['text'].str.split('_',n=1, expand=True)[0]
df['times'] = df['text'].str.split('_',n=1, expand=True)[1]
df['lat'] = df['coord'].str.split(',',n=1, expand=True)[0]
df['lat'] = df.lat.str.replace("(", "")
df['lat'] = df['lat'].astype(float)
df['lon'] = df['coord'].str.split(',',n=1, expand=True)[1]
df['lon'] = df.lon.str.replace(")", "")
df['lon'] = df['lon'].astype(float)
geometry = [Point(xy) for xy in zip(df.lon, df.lat)]
df = df.drop(['coord','text','lon', 'lat'], axis=1)
coord = gpd.GeoDataFrame(df, geometry=geometry)
coord.crs = {"init": "EPSG:4326"}
df_of_coords=coord.to_crs({"init": "EPSG:32637"})

df_of_coords['date'], df_of_coords['time'] = df_of_coords.times.str.split(' ',1).str
df_of_coords['time'] = df_of_coords.time.str.split('+',1,expand=True)
df_of_coords['hour'], df_of_coords['minuts'] = df_of_coords.time.str.split(':',1).str
df_of_coords['minuts'], df_of_coords['sek'] = df_of_coords.minuts.str.split(':',1).str
df_of_coords['hour'] = df_of_coords['hour'].astype(np.int64)
df_of_coords['minuts'] = df_of_coords['minuts'].astype(np.int64)
df_of_coords['sek'] = df_of_coords['sek'].astype(np.int64)
df_of_coords['lon'] = df_of_coords.geometry.x
df_of_coords['lat'] = df_of_coords.geometry.y
df_of_coords['lon'] = df_of_coords['lon'].astype(float)
df_of_coords['lat'] = df_of_coords['lat'].astype(float)
df_of_coords = df_of_coords.drop(['times'], axis=1)
df_of_coords = df_of_coords.reset_index().rename(columns = {'index':'nod_sequence'})
df_of_coords['line_id'] = df_of_coords['line_id'].astype(np.int64)
df_of_coords


,nod_sequence,line_id,direction,geometry,date,time,hour,minuts,sek,lon,lat
0,0,471,backward,POINT (413306.8704221835 6000056.276268307),2018-11-15,13:03:37,13,3,37,413306.870422,6.000056e+06
1,1,471,backward,POINT (413320.419880311 6000015.733834346),2018-11-15,13:03:39,13,3,39,413320.419880,6.000016e+06
2,2,471,backward,POINT (413093.1248238417 5999934.644454634),2018-11-15,13:03:49,13,3,49,413093.124824,5.999935e+06
3,3,471,backward,POINT (412685.7198990392 6000098.808732131),2018-11-15,13:04:07,13,4,7,412685.719899,6.000099e+06
4,4,471,backward,POINT (412602.4094584108 6000137.000460224),2018-11-15,13:04:10,13,4,10,412602.409458,6.000137e+06
5,5,471,backward,POINT (412461.534739274 6000202.883918932),2018-11-15,13:04:17,13,4,17,412461.534739,6.000203e+06
6,6,471,backward,POINT (412411.9849915202 6000225.971053023),2018-11-15,13:04:19,13,4,19,412411.984992,6.000226e+06
7,7,471,backward,POINT (412064.3462057851 6000413.98800804),2018-11-15,13:04:35,13,4,35,412064.346206,6.000414e+06
8,8,471,backward,POINT (411616.3707094825 6000697.327740197),2018-11-15,13:05:18,13,5,18,411616.370709,6.000697e+06
9,9,471,backward,POINT (411619.1841457729 6000700.946587143),2018-11-15,13:05:18,13,5,18,411619.184146,6.000701e+06


In [11]:
line = 471
one_route = File_to_matrix.query('line_id == @line').reset_index(drop=True)
one_route

,время,дата обкатки,route_name,stop_id,line_id,geometry
0,13:03:37,15.11.2018,1,785,471,POINT (413341.0421447341 6000080.453136149)
1,13:04:38,15.11.2018,1,757,471,POINT (412064.9700818554 6000415.868121718)
2,13:05:39,15.11.2018,1,738,471,POINT (411618.4294934076 6000699.069032389)
3,13:08:34,15.11.2018,1,708,471,POINT (411036.1134391365 6001372.012844913)
4,13:10:52,15.11.2018,1,710,471,POINT (411052.7605727652 6001747.645910882)
5,13:12:16,15.11.2018,1,700,471,POINT (410993.173081612 6002210.891407943)
6,13:13:25,15.11.2018,1,714,471,POINT (411246.5651156177 6002779.512785766)
7,13:14:43,15.11.2018,1,720,471,POINT (411319.3279929596 6003031.420630354)
8,13:15:07,15.11.2018,1,711,471,POINT (411124.7311095188 6003377.62074415)
9,13:16:30,15.11.2018,1,717,471,POINT (411295.724400138 6003681.837139054)


In [12]:
one_track = df_of_coords.query('line_id == @line').reset_index(drop=True)
unary_union = one_track.unary_union
one_track

,nod_sequence,line_id,direction,geometry,date,time,hour,minuts,sek,lon,lat
0,0,471,backward,POINT (413306.8704221835 6000056.276268307),2018-11-15,13:03:37,13,3,37,413306.870422,6.000056e+06
1,1,471,backward,POINT (413320.419880311 6000015.733834346),2018-11-15,13:03:39,13,3,39,413320.419880,6.000016e+06
2,2,471,backward,POINT (413093.1248238417 5999934.644454634),2018-11-15,13:03:49,13,3,49,413093.124824,5.999935e+06
3,3,471,backward,POINT (412685.7198990392 6000098.808732131),2018-11-15,13:04:07,13,4,7,412685.719899,6.000099e+06
4,4,471,backward,POINT (412602.4094584108 6000137.000460224),2018-11-15,13:04:10,13,4,10,412602.409458,6.000137e+06
5,5,471,backward,POINT (412461.534739274 6000202.883918932),2018-11-15,13:04:17,13,4,17,412461.534739,6.000203e+06
6,6,471,backward,POINT (412411.9849915202 6000225.971053023),2018-11-15,13:04:19,13,4,19,412411.984992,6.000226e+06
7,7,471,backward,POINT (412064.3462057851 6000413.98800804),2018-11-15,13:04:35,13,4,35,412064.346206,6.000414e+06
8,8,471,backward,POINT (411616.3707094825 6000697.327740197),2018-11-15,13:05:18,13,5,18,411616.370709,6.000697e+06
9,9,471,backward,POINT (411619.1841457729 6000700.946587143),2018-11-15,13:05:18,13,5,18,411619.184146,6.000701e+06


In [13]:
gpx_file = one_route
gpx_file['nearest_id'] = gpx_file.apply(nearest, geom_union=unary_union, df1=one_route, df2=one_track, geom1_col='geometry', src_column='nod_sequence', axis=1)
gpx_file

,время,дата обкатки,route_name,stop_id,line_id,geometry,nearest_id
0,13:03:37,15.11.2018,1,785,471,POINT (413341.0421447341 6000080.453136149),0
1,13:04:38,15.11.2018,1,757,471,POINT (412064.9700818554 6000415.868121718),7
2,13:05:39,15.11.2018,1,738,471,POINT (411618.4294934076 6000699.069032389),9
3,13:08:34,15.11.2018,1,708,471,POINT (411036.1134391365 6001372.012844913),23
4,13:10:52,15.11.2018,1,710,471,POINT (411052.7605727652 6001747.645910882),44
5,13:12:16,15.11.2018,1,700,471,POINT (410993.173081612 6002210.891407943),46
6,13:13:25,15.11.2018,1,714,471,POINT (411246.5651156177 6002779.512785766),51
7,13:14:43,15.11.2018,1,720,471,POINT (411319.3279929596 6003031.420630354),59
8,13:15:07,15.11.2018,1,711,471,POINT (411124.7311095188 6003377.62074415),68
9,13:16:30,15.11.2018,1,717,471,POINT (411295.724400138 6003681.837139054),81


In [14]:
df_of_track = one_track.merge(gpx_file[['nearest_id','stop_id']], how = 'left', left_on = 'nod_sequence', right_on = 'nearest_id')
df_of_track = df_of_track.drop_duplicates('nod_sequence', keep='first').reset_index(drop = 'True')
df_of_track = df_of_track.drop(['nearest_id'], axis=1)
df_of_track['stop_id'] = df_of_track['stop_id'].fillna(-1)
df_of_track['stop_id'] = df_of_track['stop_id'].astype(np.int64)

list_stop_group = []
list_stop_group.append(df_of_track.stop_id[0])
for i in tqdm(range(1,len(df_of_track))):    
    if df_of_track.stop_id[i] == -1:
        list_stop_group.append(list_stop_group[-1])
    else:
        list_stop_group.append(df_of_track.stop_id[i])
df_of_track['main_stop_id'] = list_stop_group
df_of_track

100%|█████████████████████████████████████| 289/289 [00:00<00:00, 57795.92it/s]


,nod_sequence,line_id,direction,geometry,date,time,hour,minuts,sek,lon,lat,stop_id,main_stop_id
0,0,471,backward,POINT (413306.8704221835 6000056.276268307),2018-11-15,13:03:37,13,3,37,413306.870422,6.000056e+06,785,785
1,1,471,backward,POINT (413320.419880311 6000015.733834346),2018-11-15,13:03:39,13,3,39,413320.419880,6.000016e+06,-1,785
2,2,471,backward,POINT (413093.1248238417 5999934.644454634),2018-11-15,13:03:49,13,3,49,413093.124824,5.999935e+06,-1,785
3,3,471,backward,POINT (412685.7198990392 6000098.808732131),2018-11-15,13:04:07,13,4,7,412685.719899,6.000099e+06,-1,785
4,4,471,backward,POINT (412602.4094584108 6000137.000460224),2018-11-15,13:04:10,13,4,10,412602.409458,6.000137e+06,-1,785
5,5,471,backward,POINT (412461.534739274 6000202.883918932),2018-11-15,13:04:17,13,4,17,412461.534739,6.000203e+06,-1,785
6,6,471,backward,POINT (412411.9849915202 6000225.971053023),2018-11-15,13:04:19,13,4,19,412411.984992,6.000226e+06,-1,785
7,7,471,backward,POINT (412064.3462057851 6000413.98800804),2018-11-15,13:04:35,13,4,35,412064.346206,6.000414e+06,757,757
8,8,471,backward,POINT (411616.3707094825 6000697.327740197),2018-11-15,13:05:18,13,5,18,411616.370709,6.000697e+06,-1,757
9,9,471,backward,POINT (411619.1841457729 6000700.946587143),2018-11-15,13:05:18,13,5,18,411619.184146,6.000701e+06,738,738


In [16]:
matrix_com = pd.merge(df_of_track, df_of_track, how='inner', on=['line_id'])
matrix_com = matrix_com.sort_values(by = ['line_id','nod_sequence_x', 'nod_sequence_y'])
matrix_com = matrix_com.rename(columns={'nod_sequence_x':'start_nod_sequence', 'geometry_x':'start_geometry',
                                        'nod_sequence_y':'end_nod_sequence', 'geometry_y':'end_geometry'})
matrix_com = matrix_com.query('start_nod_sequence < end_nod_sequence')
matrix_stages = matrix_com[matrix_com.end_nod_sequence == (matrix_com.start_nod_sequence + 1)]
matrix_stages = matrix_stages.reset_index(drop=True)
matrix_stages['long'] = 3600*(matrix_stages['hour_y'] - matrix_stages['hour_x']) + 60*(matrix_stages['minuts_y'] - matrix_stages['minuts_x']) + (matrix_stages['sek_y'] - matrix_stages['sek_x'])
matrix_stages['length'] = np.sqrt((matrix_stages['lon_y']-matrix_stages['lon_x'])**2 + (matrix_stages['lat_y']-matrix_stages['lat_x'])**2)
matrix_stages['speed'] = ((matrix_stages['length'])/(matrix_stages['long']))*3.6
matrix_stages = matrix_stages[['line_id','direction_x', 'start_nod_sequence','main_stop_id_x','start_geometry','time_x','end_nod_sequence','main_stop_id_y', 'end_geometry','time_y','long','length','speed']]
matrix_stages = matrix_stages.replace([np.inf, -np.inf], np.nan).fillna(0)
matrix_stages

,line_id,direction_x,start_nod_sequence,main_stop_id_x,start_geometry,time_x,end_nod_sequence,main_stop_id_y,end_geometry,time_y,long,length,speed
0,471,backward,0,785,POINT (413306.8704221835 6000056.276268307),13:03:37,1,785,POINT (413320.419880311 6000015.733834346),13:03:39,2,42.746658,76.943984
1,471,backward,1,785,POINT (413320.419880311 6000015.733834346),13:03:39,2,785,POINT (413093.1248238417 5999934.644454634),13:03:49,10,241.326605,86.877578
2,471,backward,2,785,POINT (413093.1248238417 5999934.644454634),13:03:49,3,785,POINT (412685.7198990392 6000098.808732131),13:04:07,18,439.236477,87.847295
3,471,backward,3,785,POINT (412685.7198990392 6000098.808732131),13:04:07,4,785,POINT (412602.4094584108 6000137.000460224),13:04:10,3,91.647355,109.976826
4,471,backward,4,785,POINT (412602.4094584108 6000137.000460224),13:04:10,5,785,POINT (412461.534739274 6000202.883918932),13:04:17,7,155.519506,79.981460
5,471,backward,5,785,POINT (412461.534739274 6000202.883918932),13:04:17,6,785,POINT (412411.9849915202 6000225.971053023),13:04:19,2,54.664369,98.395865
6,471,backward,6,785,POINT (412411.9849915202 6000225.971053023),13:04:19,7,757,POINT (412064.3462057851 6000413.98800804),13:04:35,16,395.225380,88.925710
7,471,backward,7,757,POINT (412064.3462057851 6000413.98800804),13:04:35,8,757,POINT (411616.3707094825 6000697.327740197),13:05:18,43,530.059854,44.377104
8,471,backward,8,757,POINT (411616.3707094825 6000697.327740197),13:05:18,9,738,POINT (411619.1841457729 6000700.946587143),13:05:18,0,4.583828,0.000000
9,471,backward,9,738,POINT (411619.1841457729 6000700.946587143),13:05:18,10,738,POINT (411440.1338522956 6000815.3385035),13:05:38,20,212.472394,38.245031


In [17]:
means_and_sum_1 = matrix_stages.groupby(['main_stop_id_x'])['long'].agg(['sum']).reset_index()
means_and_sum_1 = means_and_sum_1.rename(columns={'sum':'sum_seg_long'})
means_and_sum_2 = matrix_stages.groupby(['main_stop_id_x'])['length'].agg(['sum']).reset_index()
means_and_sum_2 = means_and_sum_2.rename(columns={'sum':'sum_seg_length'})
means_and_sum = means_and_sum_1.merge(means_and_sum_2, how='left', on=['main_stop_id_x'])
means_and_sum['mean_seg_speed'] = ((means_and_sum['sum_seg_length'])/(means_and_sum['sum_seg_long']))*3.6
means_and_sum = means_and_sum.replace([np.inf, -np.inf], np.nan).fillna(0)

means_and_sum_3 = matrix_stages.groupby(['line_id'])['long'].agg(['sum']).reset_index()
means_and_sum_3 = means_and_sum_3.rename(columns={'sum':'sum_long'})
means_and_sum_4 = matrix_stages.groupby(['line_id'])['length'].agg(['sum']).reset_index()
means_and_sum_4 = means_and_sum_4.rename(columns={'sum':'sum_length'})
means_and_sum_5 = means_and_sum_3.merge(means_and_sum_4, how='left', on=['line_id'])
means_and_sum_5['mean_speed'] = ((means_and_sum_5['sum_length'])/(means_and_sum_5['sum_long']))*3.6
means_and_sum_5 = means_and_sum_5.replace([np.inf, -np.inf], np.nan).fillna(0)

matrix_stages = matrix_stages.merge(means_and_sum, how='left', on=['main_stop_id_x'])
matrix_stages = matrix_stages.merge(means_and_sum_5, how='left', on=['line_id'])
matrix_stages

,line_id,direction_x,start_nod_sequence,main_stop_id_x,start_geometry,time_x,end_nod_sequence,main_stop_id_y,end_geometry,time_y,long,length,speed,sum_seg_long,sum_seg_length,mean_seg_speed,sum_long,sum_length,mean_speed
0,471,backward,0,785,POINT (413306.8704221835 6000056.276268307),13:03:37,1,785,POINT (413320.419880311 6000015.733834346),13:03:39,2,42.746658,76.943984,58,1420.366349,88.160670,4111,24116.58757,21.11888
1,471,backward,1,785,POINT (413320.419880311 6000015.733834346),13:03:39,2,785,POINT (413093.1248238417 5999934.644454634),13:03:49,10,241.326605,86.877578,58,1420.366349,88.160670,4111,24116.58757,21.11888
2,471,backward,2,785,POINT (413093.1248238417 5999934.644454634),13:03:49,3,785,POINT (412685.7198990392 6000098.808732131),13:04:07,18,439.236477,87.847295,58,1420.366349,88.160670,4111,24116.58757,21.11888
3,471,backward,3,785,POINT (412685.7198990392 6000098.808732131),13:04:07,4,785,POINT (412602.4094584108 6000137.000460224),13:04:10,3,91.647355,109.976826,58,1420.366349,88.160670,4111,24116.58757,21.11888
4,471,backward,4,785,POINT (412602.4094584108 6000137.000460224),13:04:10,5,785,POINT (412461.534739274 6000202.883918932),13:04:17,7,155.519506,79.981460,58,1420.366349,88.160670,4111,24116.58757,21.11888
5,471,backward,5,785,POINT (412461.534739274 6000202.883918932),13:04:17,6,785,POINT (412411.9849915202 6000225.971053023),13:04:19,2,54.664369,98.395865,58,1420.366349,88.160670,4111,24116.58757,21.11888
6,471,backward,6,785,POINT (412411.9849915202 6000225.971053023),13:04:19,7,757,POINT (412064.3462057851 6000413.98800804),13:04:35,16,395.225380,88.925710,58,1420.366349,88.160670,4111,24116.58757,21.11888
7,471,backward,7,757,POINT (412064.3462057851 6000413.98800804),13:04:35,8,757,POINT (411616.3707094825 6000697.327740197),13:05:18,43,530.059854,44.377104,43,534.643682,44.760866,4111,24116.58757,21.11888
8,471,backward,8,757,POINT (411616.3707094825 6000697.327740197),13:05:18,9,738,POINT (411619.1841457729 6000700.946587143),13:05:18,0,4.583828,0.000000,43,534.643682,44.760866,4111,24116.58757,21.11888
9,471,backward,9,738,POINT (411619.1841457729 6000700.946587143),13:05:18,10,738,POINT (411440.1338522956 6000815.3385035),13:05:38,20,212.472394,38.245031,194,1030.471216,19.122146,4111,24116.58757,21.11888


In [ ]:
# matrix_stages.to_csv('matrix_stages.csv', sep=';', encoding='cp1251', index=False)

In [ ]:
start_point = gpd.GeoSeries( matrix_stages['start_geometry'])
end_point = gpd.GeoSeries( matrix_stages['end_geometry'])
# Делаем списки start и end
list_start = []
for i in range(len(start_point)):
    list_start.append(list(start_point.geometry.iloc[i].coords))

list_end = []
for i in range(len(end_point)):
    list_end.append(list(end_point.geometry.iloc[i].coords))
# Делаем линии из точек старта в конец
list_line = []
for i in range(len(list_start)):
    list_line.append(LineString(list_start[i]+list_end[i]))
list_line
# Делаем GeoDataFrame
lines_from_points = gpd.GeoDataFrame()
lines_from_points['geometry'] = list_line
lines_from_points = lines_from_points.join(matrix_stages[['long','length','speed']])
lines_from_points

In [ ]:
# Рассчитываем подходящий шаг на 5 диапазонах
max_lev = lines_from_points['speed'].max()
min_lev = lines_from_points['speed'].min()
step = (max_lev - min_lev)/5
step

In [ ]:
max_lev

In [ ]:
#Делаем проверку на шаг и переприсваеваем 'speed'
steps = []
for i in range(len(lines_from_points)):
    if lines_from_points['speed'][i] < step:
        step_lev = step
        steps.append(step_lev)
    elif lines_from_points['speed'][i] < 2*step:
        step_lev = 2*step
        steps.append(step_lev)
    elif lines_from_points['speed'][i] < 3*step:
        step_lev = 3*step
        steps.append(step_lev)
    elif lines_from_points['speed'][i] < 4*step:
        step_lev = 4*step
        steps.append(step_lev)
    else:
        step_lev = 5*step
        steps.append(step_lev)
        
lines_from_points['speed_step'] = steps
lines_from_points

In [ ]:
# Делаем буфер по полю 'speed_step' (из предыдущего цикла получились 5 размеров буферов)
buf = []
for i in range(len(lines_from_points)):
    aa = lines_from_points.iloc[i].geometry.buffer((lines_from_points['speed_step'][i]))
    buf.append(aa)
spider_routes = gpd.GeoDataFrame()
spider_routes['geometry'] = buf
spider_routes = spider_routes.join(lines_from_points[['speed']])
spider_routes = spider_routes.reset_index().rename(columns = {'index':'span'})
spider_routes

In [ ]:
# spider_routes.to_file('spider_routes.shp')

In [ ]:
spider_routes.crs = {"init": "EPSG:32637"}
spider_routes=spider_routes.to_crs({"init": "EPSG:4326"})

In [ ]:
import folium

In [ ]:
bins = list(spider_routes['speed'].quantile([0, 0.25, 0.5, 0.75, 1]))

m = folium.Map(
    location=[57.627247, 39.884125],
    zoom_start=12,
    tiles='Cartodb dark_matter'
)
m.choropleth(
    geo_data=spider_routes[['span', 'geometry']].to_json(),
    name='choropleth',
    data=spider_routes[['span', 'speed']],
    key_on='feature.properties.span',
    columns=['span', 'speed'],
    fill_color='RdYlGn',
    line_weight=1,
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='type',
    bins=bins,
    reset=True,
    highlight = True
)
folium.LayerControl().add_to(m)
m.save('map.html')

In [ ]:
#  Другое

In [ ]:
orels = gpd.read_file('super-taz-work.shp', encoding='utf-8')
orels.crs = {"init": "EPSG:4326"}
# orels=orels.to_crs({"init": "EPSG:3857"})
orels

In [ ]:
bins = list(orels['workplaces'].quantile([0, 0.25, 0.5, 0.75, 1]))

m = folium.Map(
    location=[57.627247, 39.884125],
    zoom_start=12,
    tiles='Cartodb dark_matter'
)
m.choropleth(
    geo_data=orels[['new_TAZ', 'geometry']].to_json(),
    name='choropleth',
    data=orels[['new_TAZ', 'workplaces']],
    key_on='feature.properties.new_TAZ',
    columns=['new_TAZ', 'workplaces'],
    fill_color='YlGnBu',
    line_weight=1,
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='type',
    bins=bins,
    reset=True,
    highlight = True
)
folium.LayerControl().add_to(m)
m.save('map.html')

In [ ]:
folium.GeoJson(spider_routes).add_to(m)
folium.LayerControl(collapsed=False).add_to(m)
m

In [ ]:
m.save('map.html')

In [ ]:
kml_file.to_file('kml_file.shp', encoding='utf-8')